<h1 style="text-align: center;"><strong>Car Accidents Analysis</strong></h1>

<br>
<table>
    <tr>
        <td><img src="traffic1.jpg" width=400 height=800></td>
        <td><img src="traffic2.jpg" width=400 height=800></td>
    </tr>
</table>    


<h2>Project Overview</h2>

This project is a capstone project for Data analysis course on corsera, which is set up by IBM. During this project a dataset of car accidents will be analysed using Python. The dataset has many attributes which would provide us with a wide range of information on different circumstances of accidents. First, the data will be analysed to understand the attributes and which information would be useful for our analysis. Then the most suitable machine learning technique would be chosen to set up a model to predict future accidents. Finally, the model will be deployed and tested to check the accuracy of the set up model.

<h3>Libraries to be used in the analysis</h3>

In [6]:
import pandas as pd
import numpy as np

print('Hello Capstone Project Course!')

Hello Capstone Project Course!
